In [52]:
import requests
import websocket
import json
from websocket import create_connection
import time
import pandas as pd
import numpy as np
from scipy.stats import norm
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth',300)
from IPython.display import clear_output
import random
from scipy.optimize import fsolve

# from VanillasFunctions import add,barrier_check

In [53]:
# Volatility index parameters such as vol, displayed decimals
assets=pd.read_csv("assets.csv")
#API URL
master_URL = "wss://qa142.deriv.dev/websockets/v3?app_id=1007&l=EN&brand=deriv"
#api token
api_token = "ygzT3WESRSP1zBP"

In [54]:
def barrier_check (symbol, duration, duration_unit, contract_type): 
    return_json = []
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        message = json.loads(message)
        print(message)
        if 'authorize' in message:
            if duration_unit == "d":
                barrier = "10000"
            else:
                barrier = "+0"
            msg = {
                "proposal": 1,
                "amount": 60,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "USD",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            print('sendbar')
            ws.send(json.dumps(msg))
        else:
            print('here')
            amount = 60
            if 'error' in message.keys():
                barrier = message['error']['details']['barrier_choices'][int(len(message['error']['details']['barrier_choices'])/2)]
                if message['error']['details']['field'] == 'amount':
                    amount = float(message['error']['details']['max_stake'])
                    print('amount =',amount)
                    print(barrier)
                    print(contract_type)
                    print(duration)
                    print(duration_unit)
                    print(symbol)
                msg = {
                    "proposal": 1,
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "USD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                }
                print('sent')
                ws.send(json.dumps(msg))   
            else:
                return_json.append(message)
                ws.close()
    def on_close(ws, close_status_code, close_msg):
        print()             
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    barriers = return_json[-1]['proposal']['barrier_choices']
    return return_json[-1]

In [55]:
def calc_barriers (pd_series): #(spot, symbol, duration, duration_unit):   
    spot = float(pd_series.Spot)    
    symbol = pd_series.Asset    
    duration = pd_series.Start_expiry    
    duration_unit = pd_series.Duration[-1]    
    digits = int( assets[assets['Asset']== symbol].digits )    
    sigma = float( assets[assets['Asset']==symbol].IV )      
    n_digits = round( - np.log10(spot * sigma * np.sqrt(60/365/86400)) , 0) - 1    
    spot = round(spot/10**-n_digits)*10**-n_digits    
    t = (int(duration[1]) - int(duration[0])) / 365 / 86400           
    # Uncomment this section to match all API barriers to self calculated barriers
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400       
    if duration_unit != 'd':        
        deltas = np.arange(1,10,2)/10     
        real_strikes = spot * np.exp( (sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(deltas)) )    
        displayed_barriers = np.where( deltas > 0.5, np.round((real_strikes - spot)*10**digits)/10**digits , np.round((real_strikes - spot)*10**digits)/10**digits )        
    else:        
        temp_barriers = []        
        displayed_barriers = []
        central_strike = round( spot / (10 ** -n_digits) ) * (10 ** - n_digits )
        max_strike =  np.floor( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.1)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )
        min_strike =  np.ceil( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.9)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )
        strike_step_1 = round( (central_strike - min_strike) / 5 / 10 ) * 10
        strike_step_2 = round( (max_strike - central_strike) / 5 / 10 ) * 10 
        for i in range(5):
            temp_barriers.append(min_strike + strike_step_1*i)
        for i in range(5):
            temp_barriers.append(central_strike + strike_step_2*i)
        temp_barriers.append(max_strike)
        for strike in temp_barriers:
            if strike not in displayed_barriers:
                displayed_barriers.append(strike)
    return displayed_barriers       

In [56]:
def calc_ppp_ask (pd_series, t_step, s_step):
    symbol = pd_series.Asset    
    duration = pd_series.Start_expiry    
    duration_unit = pd_series.Duration[-1]    
    stake = pd_series.Stake
    vol_charge = 1.025 
    t = ( int(duration[1]) - int(duration[0]) + t_step ) / 365 / 86400      
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        print('rounded')
        t -= 2 / 365 / 86400
    r = pd_series.Discount_rate     
    contract_type = pd_series.Contract_type
    number_of_contracts = pd_series.API_PPP        
    if s_step == 0:        
        spot = float(pd_series.Spot)
        strike = pd_series.Barrier    
    else:        
        spot = float(tick_history(int(duration[0]), symbol, s_step))        
        if duration_unit != 'd':            
            strike = spot + float(pd_series.Displayed_barrier)            
        else:            
            strike = pd_series.Barrier        
    sigma = float( assets[assets['Asset']==symbol].IV )    
    if duration_unit != 'd':        
        spot_spread = float( assets[assets['Asset']==symbol].SS )    
        bs_markup = float( assets[assets['Asset']==symbol].BS )        
    else:        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )        
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))    
    if contract_type == "VANILLALONGCALL":        
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2        
    elif  contract_type == "VANILLALONGPUT":        
        phi = -1       
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2        
    bs_price = (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2)))    
    print(symbol,'s', spot, 'k',strike, r, t, vol,  phi,  'p',number_of_contracts, stake, 'd', (norm.cdf(phi*d1)))    
    return stake / ( bs_price + delta_charge + bs_markup )

In [57]:
def tick_history (tick,symbol,t):
    
    if 'R' in symbol:
        r = 2
    else:
        r = 1
        
    temp_json = []
    
    def on_open(ws):
        omsg = {
          "ticks_history": symbol,
          "adjust_start_time": 1,
          "count": 5,
          "end": tick+t*r,
          "start": 1,
          "style": "ticks"
        }
        ws.send(json.dumps(omsg))
        
    def on_message(ws, message):

        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:
            ws.send(json.dumps(msg))
            
        else:        
            temp_json.append(message)
            ws.close()
    
    def on_close(ws, close_status_code, close_msg):
        print()
        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return temp_json[-1]['history']['prices'][-1]
    

In [58]:
def get_proposal (contract_id): 
    
    contract_id = int(contract_id)
    
    return_json = []
    
    def on_open(ws):
        
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        
        ws.send(json.dumps(msg))


    def on_message(ws, message):

        message = json.loads(message)
        print(message)
        if 'authorize' in message:
            
            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_id
            }
            
            ws.send(json.dumps(msg))
            
        else:
                    
            return_json.append(message)
            
            ws.close()

    def on_close(ws, close_status_code, close_msg):
        print()

    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return return_json[-1]

In [59]:
def get_barriers (symbol, duration, duration_unit, contract_type): 
    barrier = 0
    if duration_unit == "d":
        barrier = "10000"
    else:
        barrier = "+0"
    return_json = []
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        message = json.loads(message)
        if duration_unit == "d":
            barrier = "10000"
        else:
            barrier = "+0" 
        print('\n',message)
        if 'authorize' in message:
            
            msg = {
                "proposal": 1,
                "amount": 60,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "USD",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            ws.send(json.dumps(msg))
        else:
            amount = 60
            if 'error' in message.keys():
                if message['error']['details']['field'] == 'barrier':
                    barrier = message['error']['details']['barrier_choices'][np.random.randint(len(message['error']['details']['barrier_choices']))]
                elif message['error']['details']['field'] == 'amount':
                    amount = int(message['error']['details']['min_stake']) + 5
                    barrier = message['echo_req']['barrier']
                    if message['error']['details']['min_stake'] > 2000:
                        barrier = message['error']['details']['barrier_choices'][np.random.randint(len(message['error']['details']['barrier_choices']))]
                        amount = 2

                print(amount)
                print(barrier)
                print(contract_type)
                print(duration)
                print(duration_unit)
                print(symbol)
                msg = {
                    "proposal": 1,
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "USD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                }
                print(msg)
                ws.send(json.dumps(msg))   

            else:

                return_json.append(message)
                ws.close()          
    def on_close(ws, close_status_code, close_msg):
        print()              
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    barriers = return_json[-1]['proposal']['barrier_choices']    
    if 'qa' in master_URL:        
        return barriers, 0#return_json[-1]['proposal']['pricing_args']['discount_rate']    
    else:        
        return barriers, 0

In [60]:
def get_contract (symbol, duration, duration_unit, contract_type): 
#     temp_unit = [duration_unit]
    return_json = []
    return_barrier = []   
    return_discount_rate = []
    contract_ID = []    
    def on_open(ws):        
        msg = {
            "authorize": api_token,
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        print('\n',message)
        barrier = 0        
        amount = 60        
        message = json.loads(message)        
#         print('\n',message)
#         print('here1')
        if 'authorize' in message:
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            barrier = barriers[0][np.random.randint(len(barriers[0]))]
            return_barrier.append(barrier)
            return_discount_rate.append(barriers[1])
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "USD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            ws.send(json.dumps(msg))    
        elif 'error' in message.keys():
            if 'code' in message['error'].keys():
                amount = float(message['error']['message'].split()[-1][:-1])                
                if 'ax' in message['error']['message']:
                    amount -= 5
                else: 
                    amount += 5                    
                print('here',amount)
            print('here2',amount)    
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            barrier = barriers[0][np.random.randint(len(barriers[0]))]
            return_barrier.append(barrier)
            return_discount_rate.append(barriers[1])           
            print (symbol)            
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "USD",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            print('msgcre')
            print(msg)
            ws.send(json.dumps(msg))
            print('sent')
        elif 'buy' in message:
            print('bought')
            contract_ID.append(message['buy']['contract_id'])

            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_ID[-1]
            }
            ws.send(json.dumps(msg))            
        else:
            return_json.append(message)            
            ws.close()                                          
    def on_close(ws, close_status_code, close_msg):
        print()
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()   
    return return_json[-1], return_discount_rate[-1], return_barrier[-1]

In [61]:
def calc_ppp_bid (proposal, t_step, s_step):
    symbol = proposal['proposal_open_contract']['underlying']    
    stake = float(proposal['proposal_open_contract']['bid_price'])
    vol_charge = 0.975 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['current_spot_time'] + t_step ) / 365 / 86400          
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400        
    if t > 1 / 365:        
        r = 0#proposal['proposal_open_contract']['pricing_args']['discount_rate']        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )        
    else:        
        r = 0                
        spot_spread = float( assets[assets['Asset']==symbol].SS )    
        bs_markup = float( assets[assets['Asset']==symbol].BS )    
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])                
    if s_step == 0:        
        spot = float(proposal['proposal_open_contract']['current_spot'])
        strike = float(proposal['proposal_open_contract']['barrier'])    
    else:
        spot = float(tick_history(int(proposal['proposal_open_contract']['current_spot_time']), symbol, s_step))        
        if t > 1 / 365:        
            strike = float(proposal['proposal_open_contract']['barrier'])
        else:
            strike = spot + float(proposal['proposal_open_contract']['barrier']) - float(proposal['proposal_open_contract']['current_spot'])    
    sigma = float( assets[assets['Asset']==symbol].IV )        
    if contract_type == "VANILLALONGCALL":        
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2
    elif  contract_type == "VANILLALONGPUT":        
        phi = -1        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2    
    print(spot,strike,r, t, phi,  number_of_contracts, stake)
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))
    return ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup) * number_of_contracts    

In [62]:
def sell_contract(contract_id):    
    contract_id = int(contract_id)    
    temp_json = []    
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))        
    def on_message(ws, message):
        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:            
            msg = {
              "sell": contract_id,
              "price": 0
            }
            ws.send(json.dumps(msg))           
        else:        
            temp_json.append(float(message['sell']['sold_for']))
            ws.close()    
    def on_close(ws, close_status_code, close_msg):
        print()        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()    
    return temp_json[-1]

In [63]:
def calc_ppp_sold (proposal, t_step, s_step):
    symbol = proposal['proposal_open_contract']['underlying']    
    stake = float(proposal['proposal_open_contract']['sell_price'])
    vol_charge = 0.975 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['exit_tick_time'] + t_step) / 365 / 86400  
    if t > 1 / 365:        
        r = 0#proposal['proposal_open_contract']['pricing_args']['discount_rate']        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )        
    else:        
        r = 0                
        spot_spread = float( assets[assets['Asset']==symbol].SS )
        bs_markup = float( assets[assets['Asset']==symbol].BS )    
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])        
    if s_step == 0:        
        spot = float(proposal['proposal_open_contract']['exit_tick'])
        strike = float(proposal['proposal_open_contract']['barrier'])    
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))        
        strike = float(proposal['proposal_open_contract']['barrier'])    
    sigma = float( assets[assets['Asset']==symbol].IV )
    if contract_type == "VANILLALONGCALL":        
        phi = 1        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2
    elif  contract_type == "VANILLALONGPUT":        
        phi = -1
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2        
    print('s',spot,'k',strike,'r',r,'t', t, phi,'n',  number_of_contracts,'s', stake)
    vol = sigma * vol_charge
    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))
    d2 = (d1 - vol*np.sqrt(t))    
    price = ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup)
    print(price)
    return price * number_of_contracts    

In [64]:
def calc_ppp_exp (proposal, t_step, s_step):
    stake = float(proposal['proposal_open_contract']['sell_price'])
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
    if s_step == 0:
        spot = float(proposal['proposal_open_contract']['exit_tick'])
        strike = float(proposal['proposal_open_contract']['barrier'])
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))        
#         strike = spot - float(proposal['proposal_open_contract']['exit_tick']) + float(proposal['proposal_open_contract']['barrier'])
        strike = float(proposal['proposal_open_contract']['barrier'])    
    if contract_type == "VANILLALONGCALL":
        phi = 1
    elif  contract_type == "VANILLALONGPUT":
        phi = -1
    print(spot,strike, phi,  number_of_contracts, stake)
    return max( (spot - strike) * phi * number_of_contracts, 0)

In [16]:
# barrier_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Duration', 'Start_expiry', 'API_barrier_choice', 'Calc_barrier_choices', 'json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:
    clear_output(wait = True)
    print(len(barrier_df))
    if len(barrier_df) > 30 - 1:
        break
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    duration_unit = random.choice(time_units)
    if duration_unit == 'm':        
        max_duration = 1440
    elif duration_unit == 'h':        
        max_duration = 24
    elif duration_unit == 'd':        
        max_duration = 365
    i = np.random.randint(5)    
    if i < 2:        
        duration = max_duration**i        
    else:        
        duration = np.random.randint(1,max_duration)        
    price_proposal = barrier_check(asset, duration, duration_unit, contract_type)    
    barrier_df.loc[len(barrier_df)] = [ price_proposal['echo_req']['symbol'], price_proposal['echo_req']['contract_type'], price_proposal['proposal']['spot'], \
                                        price_proposal['proposal']['spot_time'], str(price_proposal['echo_req']['duration']) + price_proposal['echo_req']['duration_unit'], \
                                        [ price_proposal['proposal']['date_start'], price_proposal['proposal']['date_expiry'] ], \
                                        [float(x) for x in price_proposal['proposal']['barrier_choices']], 0, price_proposal ]    
clear_output(wait = True)
barrier_df

,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json
0,R_25,VANILLALONGPUT,9981.9650,1689909746,1186m,"[1689909747, 1689980907]","[-150.34, -61.38, 0.71, 63.18, 154.07]",0,"{'echo_req': {'amount': 60, 'barrier': '+0.710', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'USD', 'duration': 1186, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_25'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices': ['-..."
1,R_75,VANILLALONGPUT,10053.0829,1689909748,432m,"[1689909748, 1689935668]","[-272.241, -110.935, 2.335, 116.889, 284.583]",0,"{'echo_req': {'amount': 60, 'barrier': '+2.3350', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'USD', 'duration': 432, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_75'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices': ['-..."
2,1HZ10V,VANILLALONGPUT,10046.2200,1689909750,1m,"[1689909750, 1689909810]","[-1.8, -0.7, 0.0, 0.7, 1.8]",0,"{'echo_req': {'amount': 60, 'barrier': '+0.00', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'USD', 'duration': 1, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ10V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices': ['-1...."
3,1HZ10V,VANILLALONGCALL,10046.1100,1689909752,1143m,"[1689909752, 1689978332]","[-59.8, -24.4, 0.1, 24.7, 60.4]",0,"{'echo_req': {'amount': 60, 'barrier': '+0.10', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'USD', 'duration': 1143, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ10V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices': [..."
4,1HZ10V,VANILLALONGPUT,10046.1000,1689909754,365d,"[1689909754, 1721519999]","[8890.0, 9120.0, 9350.0, 9580.0, 9810.0, 10050.0, 10340.0, 10630.0, 10920.0, 11210.0, 11480.0]",0,"{'echo_req': {'amount': 60, 'barrier': '10050.00', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'USD', 'duration': 365, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ10V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices': ..."
5,1HZ25V,VANILLALONGPUT,10157.6100,1689909809,818m,"[1689909809, 1689958889]","[-127.6, -52.1, 0.5, 53.4, 130.2]",0,"{'echo_req': {'amount': 60, 'barrier': '+0.50', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'USD', 'duration': 818, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ25V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices': ['-..."
6,R_50,VANILLALONGCALL,10134.9770,1689909810,302d,"[1689909811, 1716076799]","[6300.0, 7060.0, 7820.0, 8580.0, 9340.0, 10100.0, 12080.0, 14060.0, 16040.0, 18020.0, 20000.0]",0,"{'echo_req': {'amount': 60, 'barrier': '10100.0000', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'USD', 'duration': 302, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_50'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices':..."
7,1HZ25V,VANILLALONGCALL,10157.2300,1689909812,168d,"[1689909812, 1704499199]","[8400.0, 8760.0, 9120.0, 9480.0, 9840.0, 10200.0, 10720.0, 11240.0, 11760.0, 12280.0, 12800.0]",0,"{'echo_req': {'amount': 60, 'barrier': '10200.00', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'USD', 'duration': 168, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ25V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 60, 'barrier_choices':..."
8,1HZ75V,VANILLALONGPUT,10180.1400,1689909814,1m,"[1689909814, 1689909874]","[-13.5, -5.5, 0.0, 5.5, 13.5]",0,"{'echo_req': {'amount': 60, 'barrier': '+0.00', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'USD', 'duration': 1, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ75V'}, 'msg_type': 'proposal', 'proposal': 

In [17]:
Calc_barrier_choices = [] 
for i in barrier_df.index:    
    Calc_barrier_choices.append( calc_barriers(barrier_df.iloc[i]) )
barrier_df.Calc_barrier_choices = Calc_barrier_choices
print(len(barrier_df), 'Test cases','\nMismatches =',\
      len(barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) > 0.00000001]),\
      '\nMismatches listed below :')
barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) > 0.00000001]

30 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json


In [42]:
# PPP_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Calc_PPP','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:
    clear_output(wait=True)
    print(len(PPP_df))    
    if len(PPP_df)>15-1:
        break        
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
#     asset = 'R_50'
    duration_unit = random.choice(time_units)
#     duration_unit = 'm'
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365
    i= np.random.randint(6)    
    if i<2:        
        duration = max_duration**i        
    else:
        duration = np.random.randint(1, max_duration)
    contract = get_contract(asset, duration, duration_unit, contract_type)    
    proposal = contract[0]    
    discount_rate = contract[1]    
    displayed_barrier = contract[2]    
    PPP_df.loc[len(PPP_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, proposal]                               
#     clear_output(wait=True)
clear_output(wait=True)

PPP_df


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json
0,R_50,VANILLALONGPUT,10110.3322,1689910612,60.0,197d,7460.0000,7460.0000,"[1689910613, 1707004799]",819,0,0.16111,0,"{'echo_req': {'contract_id': 819, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '7460.0000', 'barrier_count': 1, 'bid_price': 53.07, 'buy_price': 60, 'contract_id': 819, 'contract_type': 'VANILLALONGPUT', 'currency..."
1,1HZ50V,VANILLALONGCALL,9723.3600,1689910616,60.0,1d,9300.00,9300.0000,"[1689910617, 1690070399]",839,0,0.13577,0,"{'echo_req': {'contract_id': 839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9300.00', 'barrier_count': 1, 'bid_price': 59.57, 'buy_price': 60, 'contract_id': 839, 'contract_type': 'VANILLALONGCALL', 'currency'..."
2,R_25,VANILLALONGCALL,9989.1530,1689910620,60.0,1376m,+166.110,10155.2630,"[1689910621, 1689993181]",859,0,9.13087,0,"{'echo_req': {'contract_id': 859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10155.263', 'barrier_count': 1, 'bid_price': 49.78, 'buy_price': 60, 'contract_id': 859, 'contract_type': 'VANILLALONGCALL', 'currenc..."
3,R_100,VANILLALONGPUT,10123.4200,1689910626,60.0,6d,8600.00,8600.0000,"[1689910626, 1690502399]",879,0,0.75675,0,"{'echo_req': {'contract_id': 879, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '8600.00', 'barrier_count': 1, 'bid_price': 50.54, 'buy_price': 60, 'contract_id': 879, 'contract_type': 'VANILLALONGPUT', 'currency':..."
4,R_75,VANILLALONGPUT,9994.1170,1689910630,60.0,311d,6240.0000,6240.0000,"[1689910630, 1716854399]",899,0,0.07770,0,"{'echo_req': {'contract_id': 899, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '6240.0000', 'barrier_count': 1, 'bid_price': 54.06, 'buy_price': 60, 'contract_id': 899, 'contract_type': 'VANILLALONGPUT', 'currency..."
5,1HZ25V,VANILLALONGCALL,10180.8700,1689910634,60.0,152m,-55.30,10125.5700,"[1689910634, 1689919754]",919,0,1.04277,0,"{'echo_req': {'contract_id': 919, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10125.57', 'barrier_count': 1, 'bid_price': 59.6, 'buy_price': 60, 'contract_id': 919, 'contract_type': 'VANILLALONGCALL', 'currency'..."
6,R_10,VANILLALONGPUT,10029.8900,1689910638,60.0,2h,-7.930,10021.9600,"[1689910638, 1689917838]",939,0,19.86475,0,"{'echo_req': {'contract_id': 939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10021.960', 'barrier_count': 1, 'bid_price': 54.77, 'buy_price': 60, 'contract_id': 939, 'contract_type': 'VANILLALONGPUT', 'currency..."
7,R_50,VANILLALONGCALL,10112.2953,1689910642,60.0,6h,+0.8650,10113.1603,"[1689910642, 1689932242]",959,0,1.11772,0,"{'echo_req': {'contract_id': 959, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10113.1603', 'barrier_count': 1, 'bid_price': 57.05, 'buy_price': 60, 'contract_id': 959, 'contract_type': 'VANILLALONGCALL', 'curren..."
8,R_50,VANILLALONGCALL,10113.6052,1689910646,60.0,737m,-237.7320,9875.8732,"[1689910646, 1689954866]",979,0,0.24228,0,"{'echo_req': {'contract_id': 979, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9875.8732', 'barrier_count': 1, 'bid_price': 59.6, 'buy_price': 60, 'contract_id': 979, 'contract_type': 'VANILLALONGCALL', 'currency..."
9,R_100,VANILLALONGCALL,10121.3000,1689910650,60.0,277d,17120.00,17120.0000,"[1689910650, 1713916799]",999,0

#### Mismatches below are due assumming discount rate = 0 in calculations here

In [43]:
Calc_ppp = [] 
for i in PPP_df.index:
    Calc_ppp.append( calc_ppp_ask(PPP_df.iloc[i], 0, 0) )
PPP_df.Calc_PPP = Calc_ppp
clear_output(wait = True)
print(len(PPP_df), 'Test cases','\nMismatches =',\
      len(PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,5) != 0]),\
      '\nMismatches listed below :')
df = PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,5) != 0]
df

15 Test cases 
Mismatches = 6 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json
0,R_50,VANILLALONGPUT,10110.3322,1689910612,60.0,197d,7460.0000,7460.0,"[1689910613, 1707004799]",819,0,0.16111,0.155479,"{'echo_req': {'contract_id': 819, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '7460.0000', 'barrier_count': 1, 'bid_price': 53.07, 'buy_price': 60, 'contract_id': 819, 'contract_type': 'VANILLALONGPUT', 'currency..."
1,1HZ50V,VANILLALONGCALL,9723.3600,1689910616,60.0,1d,9300.00,9300.0,"[1689910617, 1690070399]",839,0,0.13577,0.135740,"{'echo_req': {'contract_id': 839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9300.00', 'barrier_count': 1, 'bid_price': 59.57, 'buy_price': 60, 'contract_id': 839, 'contract_type': 'VANILLALONGCALL', 'currency'..."
3,R_100,VANILLALONGPUT,10123.4200,1689910626,60.0,6d,8600.00,8600.0,"[1689910626, 1690502399]",879,0,0.75675,0.756025,"{'echo_req': {'contract_id': 879, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '8600.00', 'barrier_count': 1, 'bid_price': 50.54, 'buy_price': 60, 'contract_id': 879, 'contract_type': 'VANILLALONGPUT', 'currency':..."
4,R_75,VANILLALONGPUT,9994.1170,1689910630,60.0,311d,6240.0000,6240.0,"[1689910630, 1716854399]",899,0,0.07770,0.072806,"{'echo_req': {'contract_id': 899, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '6240.0000', 'barrier_count': 1, 'bid_price': 54.06, 'buy_price': 60, 'contract_id': 899, 'contract_type': 'VANILLALONGPUT', 'currency..."
9,R_100,VANILLALONGCALL,10121.3000,1689910650,60.0,277d,17120.00,17120.0,"[1689910650, 1713916799]",999,0,0.03302,0.031239,"{'echo_req': {'contract_id': 999, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '17120.00', 'barrier_count': 1, 'bid_price': 54.57, 'buy_price': 60, 'contract_id': 999, 'contract_type': 'VANILLALONGCALL', 'currency..."
10,1HZ50V,VANILLALONGPUT,9722.1700,1689910654,60.0,365d,8920.00,8920.0,"[1689910654, 1721519999]",1019,0,0.04304,0.039680,"{'echo_req': {'contract_id': 1019, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '8920.00', 'barrier_count': 1, 'bid_price': 56.47, 'buy_price': 60, 'contract_id': 1019, 'contract_type': 'VANILLALONGPUT', 'currency..."


In [50]:
# Bid_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Bid_time', 'API_bid', 'Calc_bid','bid_prop','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:
    print(len(Bid_df))
    if len(Bid_df)>20-1:
        break
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
#     asset = 'R_10'
    duration_unit = random.choice(time_units)
    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)        
    duration = np.random.randint(max_duration/2, max_duration)
    contract = get_contract(asset, duration, duration_unit, contract_type)    
    proposal = contract[0]    
    discount_rate = contract[1]    
    displayed_barrier = contract[2]    
    Bid_df.loc[len(Bid_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, 0, 0, proposal]
    clear_output(wait=True)
Bid_df

20


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,1HZ25V,VANILLALONGCALL,10177.0200,1689911150,60.0,988m,-57.20,10119.8200,"[1689911150, 1689970430]",1639,0,0.75712,0,0,0,0,"{'echo_req': {'contract_id': 1639, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10119.82', 'barrier_count': 1, 'bid_price': 58.55, 'buy_price': 60, 'contract_id': 1639, 'contract_type': 'VANILLALONGCALL', 'curren..."
1,R_100,VANILLALONGCALL,10141.9700,1689911154,60.0,20h,+11.50,10153.4700,"[1689911154, 1689983154]",1679,0,0.31160,0,0,0,0,"{'echo_req': {'contract_id': 1679, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10153.47', 'barrier_count': 1, 'bid_price': 56.99, 'buy_price': 60, 'contract_id': 1679, 'contract_type': 'VANILLALONGCALL', 'curren..."
2,R_50,VANILLALONGCALL,10121.2875,1689911158,60.0,766m,+1.8400,10123.1275,"[1689911158, 1689957118]",1719,0,0.76835,0,0,0,0,"{'echo_req': {'contract_id': 1719, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10123.1275', 'barrier_count': 1, 'bid_price': 57.04, 'buy_price': 60, 'contract_id': 1719, 'contract_type': 'VANILLALONGCALL', 'curr..."
3,R_25,VANILLALONGPUT,9986.5300,1689911162,60.0,13h,+0.460,9986.9900,"[1689911162, 1689957962]",1759,0,1.51669,0,0,0,0,"{'echo_req': {'contract_id': 1759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9986.990', 'barrier_count': 1, 'bid_price': 57.09, 'buy_price': 60, 'contract_id': 1759, 'contract_type': 'VANILLALONGPUT', 'currenc..."
4,R_75,VANILLALONGCALL,10008.0304,1689911166,60.0,309d,6240.0000,6240.0000,"[1689911166, 1716681599]",1779,0,0.01396,0,0,0,0,"{'echo_req': {'contract_id': 1779, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '6240.0000', 'barrier_count': 1, 'bid_price': 58.94, 'buy_price': 60, 'contract_id': 1779, 'contract_type': 'VANILLALONGCALL', 'curre..."
5,R_75,VANILLALONGCALL,10005.1469,1689911170,60.0,1179m,-178.3560,9826.7909,"[1689911170, 1689981910]",1799,0,0.23959,0,0,0,0,"{'echo_req': {'contract_id': 1799, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9826.7909', 'barrier_count': 1, 'bid_price': 58.52, 'buy_price': 60, 'contract_id': 1799, 'contract_type': 'VANILLALONGCALL', 'curre..."
6,R_75,VANILLALONGPUT,10005.7616,1689911174,60.0,882m,+406.4990,10412.2606,"[1689911174, 1689964094]",1819,0,0.14209,0,0,0,0,"{'echo_req': {'contract_id': 1819, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10412.2606', 'barrier_count': 1, 'bid_price': 59.62, 'buy_price': 60, 'contract_id': 1819, 'contract_type': 'VANILLALONGPUT', 'curre..."
7,1HZ100V,VANILLALONGPUT,9347.5300,1689911178,60.0,12h,+6.40,9353.9300,"[1689911178, 1689954378]",1839,0,0.41455,0,0,0,0,"{'echo_req': {'contract_id': 1839, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9353.93', 'barrier_count': 1, 'bid_price': 57.13, 'buy_price': 60, 'contract_id': 1839, 'contract_type': 'VANILLALONGPUT', 'currency..."
8,R_100,VANILLALONGPUT,10153.2300,1689911186,141.0,259d,16740.00,16740.0000,"[1689911188, 1712361599]",1859,0,0.01753,0,0,0,0,"{'echo_req': {'contract_id': 1859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '16740.00', 'barrier_count': 1, 'bid_price': 138.18, 'buy_price': 141, 'contract_id': 1859, 'contract_type': 'VANILLALONGPUT', 'curre..."
9,R_10,VANILL

In [69]:
# calc_bid = [] 
# bid_time = []
# api_bid = []
# bid_prop = []
for i in Bid_df.index:   
    if i >= len(bid_time):       
        clear_output(wait = True)
        proposal = get_proposal(Bid_df.loc[i].ID)
        bid_time.append(proposal['proposal_open_contract']['current_spot_time'])
        api_bid.append(float(proposal['proposal_open_contract']['bid_price']))
        calc_bid.append( calc_ppp_bid(proposal, 0, 0) )       
        bid_prop.append(proposal)   
Bid_df.Bid_time = bid_time
Bid_df.API_bid = api_bid
Bid_df.Calc_bid = calc_bid
Bid_df.bid_prop = bid_prop
clear_output(wait = True)
print(len(Bid_df), 'Test cases','\nMismatches =',\
      len(Bid_df[Bid_df.API_bid - np.round(Bid_df.Calc_bid,2) != 0]),\
      '\nMismatches listed below :')
df = Bid_df[Bid_df.API_bid - np.round(Bid_df.Calc_bid,2) != 0]
df

20 Test cases 
Mismatches = 7 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
4,R_75,VANILLALONGCALL,10008.0304,1689911166,60.0,309d,6240.0000,6240.0,"[1689911166, 1716681599]",1779,0,0.01396,1689916372,60.39,64.416868,"{'echo_req': {'contract_id': 1779, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '6240.0000', 'barrier_count': 1, 'bid_price': 60.39, 'buy_price': 60, 'contract_id': 1779, 'contract_type': 'VANILLALONGCALL', 'curre...","{'echo_req': {'contract_id': 1779, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '6240.0000', 'barrier_count': 1, 'bid_price': 58.94, 'buy_price': 60, 'contract_id': 1779, 'contract_type': 'VANILLALONGCALL', 'curre..."
8,R_100,VANILLALONGPUT,10153.2300,1689911186,141.0,259d,16740.00,16740.0,"[1689911188, 1712361599]",1859,0,0.01753,1689916380,138.34,145.546375,"{'echo_req': {'contract_id': 1859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '16740.00', 'barrier_count': 1, 'bid_price': 138.34, 'buy_price': 141, 'contract_id': 1859, 'contract_type': 'VANILLALONGPUT', 'curre...","{'echo_req': {'contract_id': 1859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '16740.00', 'barrier_count': 1, 'bid_price': 138.18, 'buy_price': 141, 'contract_id': 1859, 'contract_type': 'VANILLALONGPUT', 'curre..."
9,R_10,VANILLALONGPUT,10036.0260,1689911192,60.0,277d,10540.000,10540.0,"[1689911192, 1713916799]",1879,0,0.09431,1689916380,59.63,63.036749,"{'echo_req': {'contract_id': 1879, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10540.000', 'barrier_count': 1, 'bid_price': 59.63, 'buy_price': 60, 'contract_id': 1879, 'contract_type': 'VANILLALONGPUT', 'curren...","{'echo_req': {'contract_id': 1879, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10540.000', 'barrier_count': 1, 'bid_price': 58.64, 'buy_price': 60, 'contract_id': 1879, 'contract_type': 'VANILLALONGPUT', 'curren..."
12,1HZ25V,VANILLALONGPUT,10180.8200,1689911204,60.0,241d,8520.00,8520.0,"[1689911204, 1710806399]",1939,0,0.29711,1689916464,54.56,57.143088,"{'echo_req': {'contract_id': 1939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '8520.00', 'barrier_count': 1, 'bid_price': 54.56, 'buy_price': 60, 'contract_id': 1939, 'contract_type': 'VANILLALONGPUT', 'currency...","{'echo_req': {'contract_id': 1939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '8520.00', 'barrier_count': 1, 'bid_price': 52.73, 'buy_price': 60, 'contract_id': 1939, 'contract_type': 'VANILLALONGPUT', 'currency..."
13,R_10,VANILLALONGPUT,10036.8420,1689911208,60.0,250d,9260.000,9260.0,"[1689911208, 1711583999]",1959,0,0.83715,1689916464,53.98,56.664232,"{'echo_req': {'contract_id': 1959, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9260.000', 'barrier_count': 1, 'bid_price': 53.98, 'buy_price': 60, 'contract_id': 1959, 'contract_type': 'VANILLALONGPUT', 'currenc...","{'echo_req': {'contract_id': 1959, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9260.000', 'barrier_count': 1, 'bid_price': 52.08, 'buy_price': 60, 'contract_id': 1959, 'contract_type': 'VANILLALONGPUT', 'currenc..."
17,R_10,VANILLALONGCALL,10026.3560,1689915322,60.0,217d,9300.000,9300.0,"[1689915323, 1708732799]",2419,0,0.07844,1689

In [79]:
# Sold_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:
    print(len(Sold_df))
    if len(Sold_df) > 5 - 1:
        break
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    asset = 'R_50'
    duration_unit = random.choice(['m','h','d'])
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365
    i= np.random.randint(6)
    duration = np.random.randint(max_duration/2, max_duration)
    contract = get_contract(asset, duration, duration_unit, contract_type)
    proposal = contract[0]
    discount_rate = contract[1]
    displayed_barrier = contract[2]
    Sold_df.loc[len(Sold_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), proposal]

    clear_output(wait=True)    
Sold_df

5


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,json
0,R_50,VANILLALONGPUT,10218.0553,1689916608,60.0,334d,7800.0000,7800.0000,"[1689916608, 1718841599]",3059,0,0.08178,"{'echo_req': {'contract_id': 3059, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '7800.0000', 'barrier_count': 1, 'bid_price': 54.62, 'buy_price': 60, 'contract_id': 3059, 'contract_type': 'VANILLALONGPUT', 'curren..."
1,R_50,VANILLALONGPUT,10232.3997,1689916866,60.0,23h,+343.9210,10576.3207,"[1689916866, 1689999666]",3159,0,0.16784,"{'echo_req': {'contract_id': 3159, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10576.3207', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': 60, 'contract_id': 3159, 'contract_type': 'VANILLALONGPUT', 'curre..."
2,R_50,VANILLALONGCALL,10233.3533,1689916868,60.0,207d,11700.0000,11700.0000,"[1689916870, 1707868799]",3179,0,0.05956,"{'echo_req': {'contract_id': 3179, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '11700.0000', 'barrier_count': 1, 'bid_price': 55.63, 'buy_price': 60, 'contract_id': 3179, 'contract_type': 'VANILLALONGCALL', 'curr..."
3,R_50,VANILLALONGCALL,10234.0717,1689916874,60.0,281d,15900.0000,15900.0000,"[1689916874, 1714262399]",3219,0,0.12942,"{'echo_req': {'contract_id': 3219, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '15900.0000', 'barrier_count': 1, 'bid_price': 51.93, 'buy_price': 60, 'contract_id': 3219, 'contract_type': 'VANILLALONGCALL', 'curr..."
4,R_50,VANILLALONGPUT,10235.0824,1689916876,60.0,16h,-111.3580,10123.7244,"[1689916878, 1689974478]",3239,0,1.35945,"{'echo_req': {'contract_id': 3239, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10123.7244', 'barrier_count': 1, 'bid_price': 53.22, 'buy_price': 60, 'contract_id': 3239, 'contract_type': 'VANILLALONGPUT', 'curre..."


In [80]:
calc_sold = []
sold_time = []
api_sold = []
sold_prop = []
for i in Sold_df.index:    
    if i >= len(sold_time):       
        clear_output(wait = True)        
        sell_contract(Sold_df.loc[i].ID)        
        proposal = get_proposal(Sold_df.loc[i].ID)
        sold_time.append(proposal['proposal_open_contract']['exit_tick_time'])
        api_sold.append(float(proposal['proposal_open_contract']['sell_price']))
        calc_sold.append( calc_ppp_sold(proposal, 0, 0) )        
        sold_prop.append(proposal)    
Sold_df['Sold_time'] = sold_time
Sold_df['API_sold'] = api_sold
Sold_df['Calc_sold'] = calc_sold
Sold_df['Sold_prop'] = sold_prop
clear_output(wait = True)
print(len(Sold_df), 'Test cases','\nMismatches =',\
      len(Sold_df[(Sold_df.API_sold) - np.round(Sold_df.Calc_sold,2) != 0]),\
      '\nMismatches listed below :')
df = Sold_df[(Sold_df.API_sold) - np.round(Sold_df.Calc_sold,2) != 0]
df

5 Test cases 
Mismatches = 3 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,json,Sold_time,API_sold,Calc_sold,Sold_prop
0,R_50,VANILLALONGPUT,10218.0553,1689916608,60.0,334d,7800.0000,7800.0,"[1689916608, 1718841599]",3059,0,0.08178,"{'echo_req': {'contract_id': 3059, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '7800.0000', 'barrier_count': 1, 'bid_price': 54.62, 'buy_price': 60, 'contract_id': 3059, 'contract_type': 'VANILLALONGPUT', 'curren...",1689917442,54.20,58.233564,"{'echo_req': {'contract_id': 3059, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'audit_details': {'contract_start': [{'epoch': 1689916604, 'tick': 10217.6756, 'tick_display_value': '10217.6756'}, {'epoch': 1689916606, 'tick'..."
2,R_50,VANILLALONGCALL,10233.3533,1689916868,60.0,207d,11700.0000,11700.0,"[1689916870, 1707868799]",3179,0,0.05956,"{'echo_req': {'contract_id': 3179, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '11700.0000', 'barrier_count': 1, 'bid_price': 55.63, 'buy_price': 60, 'contract_id': 3179, 'contract_type': 'VANILLALONGCALL', 'curr...",1689917448,55.91,58.067398,"{'echo_req': {'contract_id': 3179, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'audit_details': {'contract_start': [{'epoch': 1689916866, 'tick': 10232.3997, 'tick_display_value': '10232.3997'}, {'epoch': 1689916868, 'flag'..."
3,R_50,VANILLALONGCALL,10234.0717,1689916874,60.0,281d,15900.0000,15900.0,"[1689916874, 1714262399]",3219,0,0.12942,"{'echo_req': {'contract_id': 3219, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '15900.0000', 'barrier_count': 1, 'bid_price': 51.93, 'buy_price': 60, 'contract_id': 3219, 'contract_type': 'VANILLALONGCALL', 'curr...",1689917452,52.19,55.234554,"{'echo_req': {'contract_id': 3219, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'audit_details': {'contract_start': [{'epoch': 1689916870, 'tick': 10232.5425, 'tick_display_value': '10232.5425'}, {'epoch': 1689916872, 'tick'..."


In [74]:
Expired_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
                               'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'API_exp', 'Calc_exp','json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
while True:
    print(len(Expired_df))    
    if len(Expired_df)> 20-1:
        break            
    contract_type = random.choice(contract_types)
    asset = random.choice(assets.Asset)
    duration_unit = 'm'
    if duration_unit == 'm':
        max_duration = 1440
    elif duration_unit == 'h':
        max_duration = 24
    elif duration_unit == 'd':
        max_duration = 365
    i= np.random.randint(6)        
    duration = np.random.randint(1, 5)
    contract = get_contract(asset, duration, duration_unit, contract_type)    
    proposal = contract[0]    
    discount_rate = contract[1]    
    displayed_barrier = contract[2]    
    Expired_df.loc[len(Expired_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, proposal]                               
    clear_output(wait=True)
Expired_df

20


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json
0,R_100,VANILLALONGPUT,10096.340,1689917132,60.0,1m,-17.50,10078.840,"[1689917133, 1689917193]",3459,0,84.23304,0,0,"{'echo_req': {'contract_id': 3459, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10078.84', 'barrier_count': 1, 'bid_price': 49.86, 'buy_price': 60, 'contract_id': 3459, 'contract_type': 'VANILLALONGPUT', 'currenc..."
1,1HZ10V,VANILLALONGCALL,10042.710,1689917137,60.0,1m,+1.80,10044.510,"[1689917137, 1689917197]",3479,0,866.66809,0,0,"{'echo_req': {'contract_id': 3479, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10044.51', 'barrier_count': 1, 'bid_price': 49.71, 'buy_price': 60, 'contract_id': 3479, 'contract_type': 'VANILLALONGCALL', 'curren..."
2,R_25,VANILLALONGCALL,9989.821,1689917140,60.0,2m,+6.200,9996.021,"[1689917141, 1689917261]",3499,0,240.30074,0,0,"{'echo_req': {'contract_id': 3499, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9996.021', 'barrier_count': 1, 'bid_price': 49.83, 'buy_price': 60, 'contract_id': 3499, 'contract_type': 'VANILLALONGCALL', 'curren..."
3,1HZ10V,VANILLALONGPUT,10043.190,1689917144,60.0,2m,+2.50,10045.690,"[1689917144, 1689917264]",3519,0,23.05297,0,0,"{'echo_req': {'contract_id': 3519, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10045.69', 'barrier_count': 1, 'bid_price': 59.57, 'buy_price': 60, 'contract_id': 3519, 'contract_type': 'VANILLALONGPUT', 'currenc..."
4,R_25,VANILLALONGPUT,9992.156,1689917148,60.0,2m,+6.200,9998.356,"[1689917148, 1689917268]",3539,0,9.30247,0,0,"{'echo_req': {'contract_id': 3539, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9998.356', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': 60, 'contract_id': 3539, 'contract_type': 'VANILLALONGPUT', 'currenc..."
5,1HZ10V,VANILLALONGPUT,10043.420,1689917152,60.0,3m,+1.30,10044.720,"[1689917152, 1689917332]",3559,0,33.99056,0,0,"{'echo_req': {'contract_id': 3559, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10044.72', 'barrier_count': 1, 'bid_price': 58.6, 'buy_price': 60, 'contract_id': 3559, 'contract_type': 'VANILLALONGPUT', 'currency..."
6,1HZ25V,VANILLALONGCALL,10151.260,1689917158,14.0,4m,+9.00,10160.260,"[1689917158, 1689917398]",3579,0,38.84114,0,0,"{'echo_req': {'contract_id': 3579, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10160.26', 'barrier_count': 1, 'bid_price': 11.62, 'buy_price': 14, 'contract_id': 3579, 'contract_type': 'VANILLALONGCALL', 'curren..."
7,1HZ100V,VANILLALONGPUT,9560.940,1689917162,60.0,4m,+34.00,9594.940,"[1689917162, 1689917402]",3599,0,1.69709,0,0,"{'echo_req': {'contract_id': 3599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9594.94', 'barrier_count': 1, 'bid_price': 59.59, 'buy_price': 60, 'contract_id': 3599, 'contract_type': 'VANILLALONGPUT', 'currency..."
8,R_100,VANILLALONGCALL,10091.360,1689917166,60.0,1m,-7.20,10084.160,"[1689917166, 1689917226]",3619,0,6.05092,0,0,"{'echo_req': {'contract_id': 3619, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10084.16', 'barrier_count': 1, 'bid_price': 58.56, 'buy_price': 60, 'contract_id': 3619, 'contract_type': 'VANILLALONGCALL', 'curren..."
9,1HZ50V,VANILLALONGCALL,9853.830,1689917172,27.0,3m,+6.20,9860.030,"[1689917172, 1689917352]",3639,0,11.

In [84]:
calc_exp = [] 
exp_time = []
api_exp = []
for i in Expired_df.index:    
    if i >= len(exp_time):        
        clear_output(wait = True)                
        # getting proposals for expired contracts to save API payout
        proposal = get_proposal(Expired_df.loc[i].ID)        
        if proposal['proposal_open_contract']['is_sold'] == 1:   
            exp_time.append(proposal['proposal_open_contract']['date_expiry'])
            api_exp.append(float(proposal['proposal_open_contract']['sell_price']))
            # Calculating expired payout to compare againts api
            calc_exp.append( calc_ppp_exp(proposal, 0, 0) )           
        else:
            exp_time.append(0)           
            api_exp.append(0)
            calc_exp.append(0)    
Expired_df['Exp_time'] = exp_time
Expired_df['API_exp'] = api_exp
Expired_df['Calc_exp'] = calc_exp
clear_output(wait = True)
print(len(Expired_df), 'Test cases','\nMismatches =',\
      len(Expired_df[(Expired_df.API_exp) - np.round(Expired_df.Calc_exp,2) != 0]),\
      '\nMismatches listed below :')
df = Expired_df[(Expired_df.API_exp) - np.round(Expired_df.Calc_exp,2) != 0]
df

20 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json,Exp_time


In [85]:
Expired_df

,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json,Exp_time
0,R_100,VANILLALONGPUT,10096.340,1689917132,60.0,1m,-17.50,10078.840,"[1689917133, 1689917193]",3459,0,84.23304,0.00,0.000000,"{'echo_req': {'contract_id': 3459, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10078.84', 'barrier_count': 1, 'bid_price': 49.86, 'buy_price': 60, 'contract_id': 3459, 'contract_type': 'VANILLALONGPUT', 'currenc...",1689917193
1,1HZ10V,VANILLALONGCALL,10042.710,1689917137,60.0,1m,+1.80,10044.510,"[1689917137, 1689917197]",3479,0,866.66809,0.00,0.000000,"{'echo_req': {'contract_id': 3479, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10044.51', 'barrier_count': 1, 'bid_price': 49.71, 'buy_price': 60, 'contract_id': 3479, 'contract_type': 'VANILLALONGCALL', 'curren...",1689917197
2,R_25,VANILLALONGCALL,9989.821,1689917140,60.0,2m,+6.200,9996.021,"[1689917141, 1689917261]",3499,0,240.30074,0.00,0.000000,"{'echo_req': {'contract_id': 3499, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9996.021', 'barrier_count': 1, 'bid_price': 49.83, 'buy_price': 60, 'contract_id': 3499, 'contract_type': 'VANILLALONGCALL', 'curren...",1689917261
3,1HZ10V,VANILLALONGPUT,10043.190,1689917144,60.0,2m,+2.50,10045.690,"[1689917144, 1689917264]",3519,0,23.05297,75.61,75.613742,"{'echo_req': {'contract_id': 3519, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10045.69', 'barrier_count': 1, 'bid_price': 59.57, 'buy_price': 60, 'contract_id': 3519, 'contract_type': 'VANILLALONGPUT', 'currenc...",1689917264
4,R_25,VANILLALONGPUT,9992.156,1689917148,60.0,2m,+6.200,9998.356,"[1689917148, 1689917268]",3539,0,9.30247,119.67,119.666974,"{'echo_req': {'contract_id': 3539, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9998.356', 'barrier_count': 1, 'bid_price': 59.61, 'buy_price': 60, 'contract_id': 3539, 'contract_type': 'VANILLALONGPUT', 'currenc...",1689917268
5,1HZ10V,VANILLALONGPUT,10043.420,1689917152,60.0,3m,+1.30,10044.720,"[1689917152, 1689917332]",3559,0,33.99056,21.41,21.414053,"{'echo_req': {'contract_id': 3559, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10044.72', 'barrier_count': 1, 'bid_price': 58.6, 'buy_price': 60, 'contract_id': 3559, 'contract_type': 'VANILLALONGPUT', 'currency...",1689917332
6,1HZ25V,VANILLALONGCALL,10151.260,1689917158,14.0,4m,+9.00,10160.260,"[1689917158, 1689917398]",3579,0,38.84114,0.00,0.000000,"{'echo_req': {'contract_id': 3579, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10160.26', 'barrier_count': 1, 'bid_price': 11.62, 'buy_price': 14, 'contract_id': 3579, 'contract_type': 'VANILLALONGCALL', 'curren...",1689917398
7,1HZ100V,VANILLALONGPUT,9560.940,1689917162,60.0,4m,+34.00,9594.940,"[1689917162, 1689917402]",3599,0,1.69709,64.85,64.845809,"{'echo_req': {'contract_id': 3599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '9594.94', 'barrier_count': 1, 'bid_price': 59.59, 'buy_price': 60, 'contract_id': 3599, 'contract_type': 'VANILLALONGPUT', 'currency...",1689917402
8,R_100,VANILLALONGCALL,10091.360,1689917166,60.0,1m,-7.20,10084.160,"[1689917166, 1689917226]",3619,0,6.05092,49.44,49.436016,"{'echo_req': {'contract_id': 3619, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200181, 'barrier': '10084.16', 'barrier_count': 1, 'bid_price':